## Dependencies

### processing functions

In [ ]:
# function to count frequencies of variants, modified from in-house script to count sgRNA frequencies
def count_spacers_fwd(in_lib, in_fastq, column_name):

    START_KEY = "GTATGTGGTG" # identifies constant seq before mutated region to determine position
    END_KEY = "ATGTGGAAGTGCAACAATGC"  # identifies constant seq after mutated region

    dict_perfects = {x:0 for x in in_lib}
    try:
        handle = gzip.open(in_fastq, "rt")
    except:
        print("could not find fastq file")
        return

    readiter = SeqIO.parse(handle, "fastq")
    for record in readiter: # contains the seq and Qscore etc.
        read_sequence = str.upper(str(record.seq))
        KEY_REGION_START = read_sequence.find(START_KEY)
        KEY_REGION_END = read_sequence.find(END_KEY)
        if (KEY_REGION_START >=0) and (KEY_REGION_END >=0):
            guide = read_sequence[(KEY_REGION_START+10):(KEY_REGION_END)]
            if guide in dict_perfects:
                dict_perfects[guide] += 1
                
    return_df = pd.DataFrame()
    return_df['NucleotideSeq'] = dict_perfects.keys()
    return_df[column_name] = dict_perfects.values()
    return return_df

# function to add counts from each fastq (corresponding to each sample/replicate) to a column 
def add_counts_table(df_dir,fastq_dir_wildcard):
    df = pd.read_csv(df_dir,sep='\t')
    df_ori_columns = df.columns.tolist()
    for fastq in glob.glob(fastq_dir_wildcard):
        counts_df = pd.DataFrame()
        counts_df = count_spacers_fwd(df['NucleotideSeq'],fastq,fastq.split('/')[-1].split('.')[0].split('_')[0])
        df = pd.merge(df,counts_df,on='NucleotideSeq')
    columns_sort = df_ori_columns + ['mCherry-R1','mCherry-R2','mCherry-R3','GFPpos-R1','GFPpos-R2','GFPpos-R3','GFPneg-R1','GFPneg-R2','GFPneg-R3']
    return df[columns_sort]

# function to get normalized log-fold changes by first converting to reads per million and adding a pseudocount of 1 and log-normalizing to mCherry (unsorted)
def normalize_counts(df):
    df_normalized = df.copy()
    for column in df.columns.tolist()[4:]:
        df_normalized[column+'_normalized'] = df_normalized[column].apply(lambda x: np.log2((x * 10000000 / df_normalized[column].sum()) + 1))
    for i in range(13,16):
        df_normalized[df_normalized.columns.tolist()[i+3][:-11]+'_mCherry_normalized']  = df_normalized[df_normalized.columns.tolist()[i+3]]-df_normalized[df_normalized.columns.tolist()[i]]
        df_normalized[df_normalized.columns.tolist()[i+6][:-11]+'_mCherry_normalized']  = df_normalized[df_normalized.columns.tolist()[i+6]]-df_normalized[df_normalized.columns.tolist()[i]]
    return df_normalized

# function to average 
def average_reps(normalized_df):
    averaged_df = normalized_df[normalized_df.columns.tolist()[0:4]]
    for i in range(22,24,1):
        column_name = normalized_df.columns.tolist()[i].split('_')[0].split('-')[0] + '_normalized_average'
        #print(column_name)
        #print([normalized_df.columns.tolist()[i]]+[normalized_df.columns.tolist()[i+2]]+[normalized_df.columns.tolist()[i+4]])
        averaged_df[column_name] = normalized_df[[normalized_df.columns.tolist()[i]]+[normalized_df.columns.tolist()[i+2]]+[normalized_df.columns.tolist()[i+4]]].mean(axis=1)
    return(averaged_df)

## example lines of code to process fastqs and obtain processed and normalized values for each variant

In [ ]:
in_lib = #csv of sequences in library
in_fastq = #fastq files (can use wildcard if desired)
DMS_df = add_counts_table(in_lib,in_fastq)
DMS_normalized = normalize_counts(DMS_df)
DMS_averaged = average_reps(DMS_normalized)

## example lines of code to construct matrices summarizing single and double substitutions and insertions

In [ ]:
aa_list = ['E','D','R','H','K','F','Y','W','S','Q','T','N','C','P','A','G','M','V','I','L']
WT_seq = 'GGSIPRR'

In [ ]:
# constructing single subtitution matrix
MutSing_arr = np.zeros((20,7)) #20 amino acids, length of mutated span is 7 residues

for y in range(len(WT_seq)): #iterate through each position in sequence
    for x, aa_1 in enumerate(aa_list): #mutate each position to each other amino acid
        if y == len(WT_seq)-1: #if at end of sequence
            hit_1 = WT_seq[:y] + aa_1
        else: hit_1 = WT_seq[:y] + aa_1 + WT_seq[(y+1):]
        MutSing_arr[x,y] = DMS_plotting[DMS_plotting['Amino_acid_Sequence']==hit_1]['GFPneg_normalized_average'].item()

In [ ]:
# constructing double substitution matrix
MutDoub_dict = dict() #constructing a dictionary that store will store a 20 by 7 matrix under the key of the "first" substitution

for aa_1 in aa_list: #creating empty arrays
    MutDoub_dict[aa_1] = np.zeros((20,6))

for y in range(len(WT_seq)-1): #iterate through each position in sequence
    for aa_1 in aa_list: #keep first aa constant
        for x, aa_2 in enumerate(aa_list): #while iterate through second aa
            mut_seq = WT_seq[:y] + aa_1 + aa_2 + WT_seq[y+2:]
            MutDoub_dict[aa_1][x,y] = DMS_plotting[(DMS_plotting['Amino_acid_Sequence']==mut_seq)]['GFPneg_normalized_average'].item()

In [ ]:
# constructing single insertion matrix
InsSing_arr = np.zeros((20,7)) #20 amino acids, length of mutated span is 7 residues and insertions starts between aa 1 and 2 in sequence

for y in range(1,len(WT_seq)+1): #iterate through each position in sequence
    for x, aa_1 in enumerate(aa_list):
        if y == len(WT_seq): #if at end of sequence
            hit_1 = WT_seq + aa_1 #add insertion to the end of sequence
        else: hit_1 = WT_seq[:y] + aa_1 + WT_seq[y:] #otherwise, insert between       
        InsSing_arr[x,y-1] = DMS_plotting[DMS_plotting['Amino_acid_Sequence']==hit_1]['GFPneg_normalized_average'].item()

In [ ]:
# constructing double insertion matrix
InsDoub_dict = dict() #constructing a dictionary that store will store a 20 by 7 matrix under the key of the "first" substitution

for aa in aa_list: #creating empty arrays
    InsDoub_dict[aa] = np.zeros((20,7))

for y in range(1,len(WT_seq)+1): #iterate through each position in sequence
    for aa_1 in aa_list: #keep first aa constant
        for x, aa_2 in enumerate(aa_list): #while iterate through second aa
            if y == len(WT_seq): #if at end of seqeunce 
                hit_1 = WT_seq + aa_1 + aa_2
            else: hit_1 = WT_seq[:y] + aa_1 + aa_2 + WT_seq[y:]
            InsDoub_dict[aa_1][x,y-1] = DMS_plotting[DMS_plotting['Amino_acid_Sequence']==hit_1]['GFPneg_normalized_average'].item()